#### 1. Fetch price, volume, liquidity proxy (from Binance via CCXT — public endpoint)

In [3]:
import ccxt
import pandas as pd
from datetime import datetime
import time

exchange = ccxt.binance()

def fetch_ohlcv(symbol, timeframe='1d', since=None, limit=1000):
    if since is None:
        since = exchange.parse8601('2024-01-01T00:00:00Z')
    data = []
    while True:
        ohlcv = exchange.fetch_ohlcv(symbol, timeframe, since, limit)
        if not ohlcv:
            break
        data += ohlcv
        since = ohlcv[-1][0] + 1  # next start
        time.sleep(0.5)           # polite rate limit
        if len(ohlcv) < limit:
            break
    df = pd.DataFrame(data, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
    df['date'] = pd.to_datetime(df['timestamp'], unit='ms').dt.date
    return df.set_index('date')

btc_price = fetch_ohlcv('BTC/USDT')
eth_price = fetch_ohlcv('ETH/USDT')

btc_price.to_csv('btc_price_volume_2024_2026.csv')
eth_price.to_csv('eth_price_volume_2024_2026.csv')

In [4]:
btc_price.head()

,timestamp,open,high,low,close,volume
date,,,,,,
2024-01-01,1704067200000,42283.58,44184.10,42180.77,44179.55,27174.29903
2024-01-02,1704153600000,44179.55,45879.63,44148.34,44946.91,65146.40661
2024-01-03,1704240000000,44946.91,45500.00,40750.00,42845.23,81194.55173
2024-01-04,1704326400000,42845.23,44729.58,42613.77,44151.10,48038.06334
2024-01-05,1704412800000,44151.10,44357.46,42450.00,44145.11,48075.25327


In [6]:
eth_price.tail()

,timestamp,open,high,low,close,volume
date,,,,,,
2026-02-12,1770854400000,1941.19,2001.42,1897.24,1947.85,427567.1469
2026-02-13,1770940800000,1947.85,2073.68,1924.72,2048.72,425649.7698
2026-02-14,1771027200000,2048.72,2107.67,2042.37,2086.59,220677.2355
2026-02-15,1771113600000,2086.59,2103.32,1928.88,1966.58,834778.0952
2026-02-16,1771200000000,1966.59,1981.20,1948.76,1972.65,81240.1734


#### 2. Fetch BTC on-chain from Blockchain.com (JSON API — very reliable & free)

In [7]:
import requests

def fetch_blockchain_com(metric, timespan='all', roll_avg=None):
    url = f"https://api.blockchain.info/charts/{metric}?timespan={timespan}&format=json"
    if roll_avg:
        url += f"&rollingAverage={roll_avg}"
    r = requests.get(url)
    data = r.json()['values']
    df = pd.DataFrame(data)
    df['x'] = pd.to_datetime(df['x'], unit='s').dt.date
    df = df.rename(columns={'x': 'date', 'y': metric}).set_index('date')
    return df

# Popular BTC metrics
btc_active_addresses = fetch_blockchain_com('n-unique-addresses')
btc_tx_volume_btc   = fetch_blockchain_com('estimated-transaction-volume')
btc_hash_rate       = fetch_blockchain_com('hash-rate')
btc_tx_count        = fetch_blockchain_com('n-transactions')

#### 3. ETH basic on-chain (CoinGecko for supply)

In [9]:
import ccxt
import pandas as pd
from datetime import datetime

exchange = ccxt.binance()

def fetch_ohlcv_full(symbol='ETH/USDT', timeframe='1d', since_str='2024-01-01T00:00:00Z'):
    since = exchange.parse8601(since_str)
    data = []
    while True:
        ohlcv = exchange.fetch_ohlcv(symbol, timeframe, since, limit=1000)
        if not ohlcv:
            break
        data.extend(ohlcv)
        since = ohlcv[-1][0] + 1
        if len(ohlcv) < 1000:
            break
    df = pd.DataFrame(data, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
    df['date'] = pd.to_datetime(df['timestamp'], unit='ms').dt.date
    df = df.set_index('date')
    return df[['close', 'volume']]  # price + trading volume

eth_binance = fetch_ohlcv_full('ETH/USDT', '1d', '2024-01-01T00:00:00Z')
eth_binance.rename(columns={'close': 'price_usd', 'volume': 'trading_volume'}, inplace=True)
eth_binance.to_csv('eth_binance_2024_2026.csv')

print(eth_binance.head())

            price_usd  trading_volume
date                                 
2024-01-01    2352.04     216702.6914
2024-01-02    2355.34     458041.6642
2024-01-03    2209.72     798729.4869
2024-01-04    2267.11     448844.0550
2024-01-05    2268.78     405568.0006
